In [46]:
from multiprocessing import Manager, Process
import Pyro4
import networkx as nx 
from time import time, sleep


@Pyro4.expose
class NetworkxServer(object):
    def __init__(self, neighbors_fpath):
        tic = time()
        print("Loading the graph:", neighbors_fpath)
        self.G = nx.read_edgelist(
           neighbors_fpath,
           nodetype=str,
           delimiter="\t",
           data=(('weight',float),))
        
        print("\nLoaded in {:f} sec.".format(time() - tic))
                   
    def get_neighbors(self, node):
        return self.G[node]
    
    def get_node(self, node):
        return self.G.nodes[node]
    
    def get_edge(self, node_i, node_j):
        return self.G[node_i][node_j]    
    
    def get_node_list(self):
        return self.G.nodes.keys()
    

def run_pyro_daemon(neighbors_fpath):
    networkx_server = NetworkxServer(neighbors_fpath)
    daemon = Pyro4.Daemon()   
    uri = daemon.register(networkx_server)
    d["uri"] = uri
    daemon.requestLoop()
    

d = Manager().dict()


class NetworxServerManager(object):
    def __init__(self, neighbors_fpath):
        
        self._graph_server = self._graph_server = Process(
                name='NetworkX RPC server',
                target=run_pyro_daemon,
                args=(neighbors_fpath,))
        self._graph_server.daemon = True
        self._graph_server.start()
        
        global d
        d["uri"] = ""
        while(d["uri"] == ""):
            sleep(1)
            print(".", end="")
            
        self._uri = d["uri"]
        print("\nURI:", self._uri)        
        self.graph = Pyro4.Proxy(self._uri) 
        
    def stop(self):
        self._graph_server.terminate()        
    
neighbors_fpath = "/home/panchenko/sensegram/model/text8.graph"
s = NetworxServerManager(neighbors_fpath)

Loading the graph: /home/panchenko/sensegram/model/text8.graph
.......................................................................
Loaded in 71.306148 sec.
.
URI: PYRO:obj_edffe93781594512b94099b17804be7c@localhost:37503


In [47]:
s.graph.get_neighbors("the")

{'_atlas': {'a': {'weight': 0.901999},
  'aba': {'weight': 0.336302},
  'about': {'weight': 0.835913},
  'abu_dhabi': {'weight': 0.281319},
  'accidently': {'weight': 0.280137},
  'ad_police': {'weight': 0.307404},
  'adaptable': {'weight': 0.288977},
  'adults': {'weight': 0.456392},
  'advances': {'weight': 0.295795},
  'aerosol': {'weight': 0.328586},
  'age': {'weight': 0.386564},
  'albedo': {'weight': 0.893342},
  'all': {'weight': 0.528141},
  'also': {'weight': 0.665255},
  'although': {'weight': 0.526195},
  'an': {'weight': 0.764593},
  'an_alternative': {'weight': 0.305604},
  'an_amorphous': {'weight': 0.290297},
  'an_important': {'weight': 0.28571},
  'anarchism': {'weight': 0.801308},
  'anarchist': {'weight': 0.594362},
  'anarchists': {'weight': 0.682834},
  'and': {'weight': 0.925761},
  'another': {'weight': 0.383097},
  'ansi': {'weight': 0.313347},
  'anybody': {'weight': 0.299141},
  'apostles': {'weight': 0.309886},
  'apple_macintosh': {'weight': 0.302323},
  'a